In [5]:
import pandas as pd
from transformers import pipeline, RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from datasets import Dataset

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load and preprocess the data
file_path = 'scrapped_dhl.csv'  # Update this with the path to your file
df = pd.read_csv(file_path)

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

df['Processed_Review'] = df['Review'].apply(preprocess_text)

# Map rating scores to numeric values (1 to 5)
df['label'] = df['Rating'] - 1  # Convert to 0-indexed for model training

# Prepare dataset for training
dataset = Dataset.from_pandas(df[['Processed_Review', 'label']])
dataset = dataset.rename_column("Processed_Review", "text")

# Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)  # 5 labels for ratings 1 to 5

# Tokenize the dataset with padding and truncation
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Create a prediction pipeline
rating_pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, return_all_scores=True)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10930 [00:00<?, ? examples/s]

C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.206200,0.165459,0.970174,0.958476,0.947060,0.970174
2,0.155400,0.141575,0.973010,0.961596,0.950733,0.973010
3,0.130400,0.104371,0.976304,0.968832,0.964910,0.976304


C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


{'eval_loss': 0.10437119752168655, 'eval_accuracy': 0.9763037511436413, 'eval_f1': 0.9688320702458673, 'eval_precision': 0.9649103790616276, 'eval_recall': 0.9763037511436413, 'eval_runtime': 1078.9202, 'eval_samples_per_second': 10.13, 'eval_steps_per_second': 1.267, 'epoch': 3.0}


TypeError: '>' not supported between instances of 'dict' and 'dict'

In [7]:
# Define a custom rating prediction function
def predict_rating(text):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    outputs = model(**inputs)
    scores = outputs.logits.detach().cpu().numpy()[0]
    rating = scores.argmax() + 1  # Convert back to 1-indexed rating
    return rating

# Apply the rating prediction to each processed review
df['Predicted_Rating'] = df['Processed_Review'].apply(predict_rating)

# Map predicted ratings to numeric values
df['Predicted_Rating_Num'] = df['Predicted_Rating']

# Calculate and print the confusion matrix and accuracy
cm = confusion_matrix(df['Rating'], df['Predicted_Rating_Num'])
accuracy = accuracy_score(df['Rating'], df['Predicted_Rating_Num'])

print(f"Confusion Matrix:\n{cm}")
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

# Save the results to a new CSV file
output_csv_path = 'RoBERTa_Rating_Prediction_Results.csv'
df.to_csv(output_csv_path, index=False)

print(f"File saved as {output_csv_path}")

Confusion Matrix:
[[10221     0     0     0     0]
 [  103    15     0    11    17]
 [   17     8     0     9    16]
 [   15     7     0    12    31]
 [   21     3     0     1   423]]
Overall Accuracy: 97.63%
File saved as RoBERTa_Rating_Prediction_Results.csv


In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load and preprocess the data
file_path = 'scrapped_dhl.csv'  # Update this with the path to your file
df = pd.read_csv(file_path)

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

df['Processed_Review'] = df['Review'].apply(preprocess_text)

# Prepare features and labels
X = df['Processed_Review']
y = df['Rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_tfidf, y_train)
y_pred_log_reg = log_reg.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print(classification_report(y_test, y_pred_log_reg))

# Support Vector Machine
svm = SVC()
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_test_tfidf)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_tfidf, y_train)
y_pred_rf = rf.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Logistic Regression Accuracy: 0.944647758462946
              precision    recall  f1-score   support

           1       0.95      1.00      0.97      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.92      0.42      0.58        81

    accuracy                           0.94      2186
   macro avg       0.37      0.28      0.31      2186
weighted avg       0.91      0.94      0.92      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVM Accuracy: 0.9423604757548033
              precision    recall  f1-score   support

           1       0.94      1.00      0.97      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.88      0.37      0.52        81

    accuracy                           0.94      2186
   macro avg       0.37      0.27      0.30      2186
weighted avg       0.91      0.94      0.92      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest Accuracy: 0.938700823421775
              precision    recall  f1-score   support

           1       0.94      1.00      0.97      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.76      0.32      0.45        81

    accuracy                           0.94      2186
   macro avg       0.34      0.26      0.28      2186
weighted avg       0.90      0.94      0.92      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load and preprocess the data
file_path = 'scrapped_dhl.csv'  # Update this with the path to your file
df = pd.read_csv(file_path)

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

df['Processed_Review'] = df['Review'].apply(preprocess_text)

# Prepare features and labels
X = df['Processed_Review']
y = df['Rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define hyperparameter grid for Logistic Regression
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear']
}

# Define hyperparameter grid for SVM
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Define hyperparameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Perform Grid Search for Logistic Regression
grid_search_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train_tfidf, y_train)
best_lr = grid_search_lr.best_estimator_
y_pred_lr = best_lr.predict(X_test_tfidf)
print("Best Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Best Logistic Regression Params:", grid_search_lr.best_params_)
print(classification_report(y_test, y_pred_lr))

# Perform Grid Search for SVM
grid_search_svm = GridSearchCV(SVC(), param_grid_svm, cv=5, scoring='accuracy')
grid_search_svm.fit(X_train_tfidf, y_train)
best_svm = grid_search_svm.best_estimator_
y_pred_svm = best_svm.predict(X_test_tfidf)
print("Best SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Best SVM Params:", grid_search_svm.best_params_)
print(classification_report(y_test, y_pred_svm))

# Perform Grid Search for Random Forest
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='accuracy')
grid_search_rf.fit(X_train_tfidf, y_train)
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_tfidf)
print("Best Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Best Random Forest Params:", grid_search_rf.best_params_)
print(classification_report(y_test, y_pred_rf))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Best Logistic Regression Accuracy: 0.9492223238792314
Best Logistic Regression Params: {'C': 100, 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           1       0.96      1.00      0.98      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.76      0.64      0.70        81

    accuracy                           0.95      2186
   macro avg       0.34      0.33      0.33      2186
weighted avg       0.92      0.95      0.93      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best SVM Accuracy: 0.9483074107959744
Best SVM Params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
              precision    recall  f1-score   support

           1       0.96      0.99      0.98      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.76      0.65      0.70        81

    accuracy                           0.95      2186
   macro avg       0.34      0.33      0.34      2186
weighted avg       0.92      0.95      0.93      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Random Forest Accuracy: 0.9382433668801464
Best Random Forest Params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
              precision    recall  f1-score   support

           1       0.94      1.00      0.97      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       0.79      0.32      0.46        81

    accuracy                           0.94      2186
   macro avg       0.35      0.26      0.28      2186
weighted avg       0.90      0.94      0.92      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load and preprocess the data
file_path = 'scrapped_dhl.csv'  # Update this with the path to your file
df = pd.read_csv(file_path)

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

df['Processed_Review'] = df['Review'].apply(preprocess_text)

# Prepare features and labels
X = df['Processed_Review']
y = df['Rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
from sklearn.naive_bayes import MultinomialNB

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Multinomial Naive Bayes model
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)
y_pred_mnb = mnb.predict(X_test_tfidf)

# Evaluate the MNB model
print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_mnb))
print(classification_report(y_test, y_pred_mnb))


Multinomial Naive Bayes Accuracy: 0.929551692589204
              precision    recall  f1-score   support

           1       0.93      1.00      0.96      2031
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        16
           4       0.00      0.00      0.00        17
           5       1.00      0.01      0.02        81

    accuracy                           0.93      2186
   macro avg       0.39      0.20      0.20      2186
weighted avg       0.90      0.93      0.90      2186



C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
import gensim.downloader as api
import numpy as np
try:
    from numpy import triu
except ImportError:
    from scipy.linalg import triu

# Load word2vec and GloVe embeddings
word2vec_model = api.load("word2vec-google-news-300")
glove_model = api.load("glove-wiki-gigaword-300")

# Function to create embedding matrix
def create_embedding_matrix(model, tokenizer, vocab_size, embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if i < vocab_size:
            try:
                embedding_vector = model[word]
                embedding_matrix[i] = embedding_vector
            except KeyError:
                continue
    return embedding_matrix

# Tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100  # Adjust as necessary
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Create embedding matrices
embedding_dim = 300
word2vec_embedding_matrix = create_embedding_matrix(word2vec_model, tokenizer, vocab_size, embedding_dim)
glove_embedding_matrix = create_embedding_matrix(glove_model, tokenizer, vocab_size, embedding_dim)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

def build_clstm_model(embedding_matrix, vocab_size, embedding_dim, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))  # Update the output layer to match your number of classes (5)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate CLSTM with Word2Vec
clstm_word2vec = build_clstm_model(word2vec_embedding_matrix, vocab_size, embedding_dim, max_length)
clstm_word2vec.fit(X_train_padded, y_train, epochs=5, verbose=1, validation_split=0.2)  # Adjust epochs as necessary
y_pred_clstm_word2vec = clstm_word2vec.predict(X_test_padded).argmax(axis=1) + 1

print("CLSTM with Word2Vec Accuracy:", accuracy_score(y_test, y_pred_clstm_word2vec))
print(classification_report(y_test, y_pred_clstm_word2vec))

# Train and evaluate CLSTM with GloVe
clstm_glove = build_clstm_model(glove_embedding_matrix, vocab_size, embedding_dim, max_length)
clstm_glove.fit(X_train_padded, y_train, epochs=5, verbose=1, validation_split=0.2)  # Adjust epochs as necessary
y_pred_clstm_glove = clstm_glove.predict(X_test_padded).argmax(axis=1) + 1

print("CLSTM with GloVe Accuracy:", accuracy_score(y_test, y_pred_clstm_glove))
print(classification_report(y_test, y_pred_clstm_glove))


Epoch 1/5


C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\asyncio\windows_events.py", line 322, in run_forever

  File "C:\Users\krish\anaconda3\envs\nlp_env\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\